In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ag_news")
dataset

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
import pandas as pd
df_test = pd.DataFrame(dataset['test'])
df_train = pd.DataFrame(dataset['train'])
df_train

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [ ]:
# need reselect train and test
# random + each label (4000)

new_train = df_train.groupby('label').apply(lambda x: x.sample(n=4000)).reset_index(drop=True)

new_train = new_train.replace(to_replace=r'[^\w\s]', value='', regex=True)


In [ ]:
import numpy as np
x_train_df = new_train["text"]
x_train = x_train_df.to_numpy()
x_train = np.vectorize(str.lower)(x_train)
y_train_df = new_train["label"]
y_train = y_train_df.to_numpy()

x_test_df = df_test["text"]
x_test = x_test_df.to_numpy()
x_test = np.vectorize(str.lower)(x_test)
y_test_df = df_test["label"]
y_test = y_test_df.to_numpy()
# need lowercased + remove some symbols
# remove stopwprds


In [ ]:
all_words_dict = {}
for word_list in x_train:
  #print(word_list)
  words = word_list.split()
  for word in words:
    #print(word)
    if all_words_dict.get(word) != None:
      all_words_dict[word] += 1
    else:
      all_words_dict[word] = 1
#all_words_dict
all_words_tuple_list = sorted(all_words_dict.items(), key=lambda f: f[1], reverse=True)  # decending
all_words_list = list(list(zip(*all_words_tuple_list))[0])


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# no need data cleaning
# NLTK stopwords list
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
print(len(stopwords))

179


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# stopword from https://mimno.github.io/Mallet/import-stoplist.html
path = "/content/gdrive/MyDrive/Colab Notebooks/641_project/svm/mallet_en_stoplist.txt"
my_file = open(path, "r")
data = my_file.read()
stopwords_set = data.split("\n")
print(stopwords_set)
my_file.close()

['a', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', 'around', 'as', 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'b', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'c', 'came', 'can', 'cannot', 'cant', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', 'co', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', 'course', 'currentl

In [ ]:
def words_dict(all_words_list, deleteN, stopwords_set=set()):
  feature_words = []
  n = 1
  for t in range(deleteN, len(all_words_list), 1):
    if n > 100:  # demension
      break
        # non-number, non stopwords
    if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set :
      feature_words.append(all_words_list[t])
      n += 1
  print("feature words：",feature_words)
  return feature_words


In [ ]:
feature_words = words_dict(all_words_list, 0, stopwords_set)

feature words： ['39s', 'reuters', 'ap', 'oil', 'monday', 'world', 'wednesday', 'company', 'thursday', 'tuesday', 'friday', 'year', 'yesterday', 'president', 'york', 'iraq', 'million', 'united', 'week', 'microsoft', 'prices', 'game', 'security', 'corp', 'years', 'sunday', 'time', 'today', 'group', 'people', 'government', 'team', 'afp', 'quot', 'percent', 'win', 'back', 'software', 'deal', 'internet', 'night', 'season', 'minister', 'saturday', 'business', 'market', 'state', 'billion', 'top', 'announced', 'international', 'china', 'bush', 'officials', 'washington', 'sales', 'victory', 'news', 'day', 'killed', 'service', 'end', 'open', 'stocks', 'plans', 'states', 'city', 'court', 'profit', 'chief', 'home', 'report', 'technology', 'lead', 'european', 'online', 'cup', 'red', 'record', 'set', 'talks', 'national', 'american', 'league', 'computer', 'space', 'federal', 'music', 'made', 'bank', 'network', 'san', 'search', 'reported', 'leader', 'india', 'web', 'shares', 'prime', '39t']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def text_feature_(text, feature_words):
    text_words = text.split()
    features = [1 if word in text_words else 0 for word in feature_words]
    return features

In [ ]:
def text_features(train_data_list, test_data_list, feature_words):
  
  train_feature_list = [text_feature_(text, feature_words) for text in train_data_list]
  test_feature_list = [text_feature_(text, feature_words) for text in test_data_list]
  print("feature vecotr：",train_feature_list,"\n")
  return train_feature_list, test_feature_list


In [ ]:
train_vect, test_vect = text_features(x_train,x_test,feature_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#train_vect
np.count_nonzero(train_vect)

50274

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import time
clf = LinearSVC()
svm = CalibratedClassifierCV(clf)



In [ ]:
t0 = time.process_time()
classifier = svm.fit(train_vect, y_train)
t1 = time.process_time()
training_time = t1 - t0
y_pred = classifier.predict(test_vect)
y_true = y_test
print("True Labels：", y_true)
print("Predicted Labels：", y_pred)
C = confusion_matrix(y_true, y_pred)
print("confusion matrix:\n",C, end='\n')
print("accuracy score:\n", accuracy_score(y_true, y_pred))
print("training time: ", training_time)


True Labels： [2 3 3 ... 1 2 2]
Predicted Labels： [2 3 3 ... 1 0 3]
confusion matrix:
 [[1238  192  104  366]
 [ 166 1256   52  426]
 [ 243  155 1038  464]
 [ 165  163  186 1386]]
accuracy score:
 0.6471052631578947
training time:  4.058219819999998


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.65      0.67      1900
           1       0.71      0.66      0.69      1900
           2       0.75      0.55      0.63      1900
           3       0.52      0.73      0.61      1900

    accuracy                           0.65      7600
   macro avg       0.67      0.65      0.65      7600
weighted avg       0.67      0.65      0.65      7600

[[1238  192  104  366]
 [ 166 1256   52  426]
 [ 243  155 1038  464]
 [ 165  163  186 1386]]
